In [17]:
MODEL = "gemma2"
EMBEDDING_MODEL = "nomic-embed-text"

In [18]:
from  langchain_ollama.chat_models import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings

In [19]:
model = ChatOllama(
  model=MODEL,
  temperature=0
) 

embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

In [20]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
# chain.invoke("what is the solar system?")

In [21]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("2008_Mumbai_attacks.pdf")
pages = loader.load_and_split()
# pages

In [22]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [7]:
chain = prompt | model | parser

In [8]:
chain.invoke({
  "context": "I was given name dishant miyani and i was born in surat gujarat india",
  "question": "where was dishant born?"
})

'Surat, Gujarat, India \n'

In [9]:
from langchain.vectorstores import DocArrayInMemorySearch

vector_store = DocArrayInMemorySearch.from_documents(
  pages,
  embedding=embeddings
)

/home/dishant/programs/RAG/.venv/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [14]:
retriever = vector_store.as_retriever()

retriever.invoke("which cafe was attacked?")

[Document(metadata={'source': '2008_Mumbai_attacks.pdf', 'page': 26}, page_content='center" (http://edition.cnn.com/2008/WORLD/asiapcf/11/27/india.attacks/index.html). CNN.\nArchived (https://web.archive.org/web/20081202132254/http://edition.cnn.com/2008/WORL\nD/asiapcf/11/27/india.attacks/index.html) from the original on 2 December 2008. Retrieved\n28 November 2008.'),
 Document(metadata={'source': '2008_Mumbai_attacks.pdf', 'page': 28}, page_content='CNN. 28 November 2008. Archived (https://web.archive.org/web/20081201083956/http://edi\ntion.cnn.com/2008/WORLD/asiapcf/11/28/india.attacks/) from the original on 1 December\n2008. Retrieved 7 March 2009.'),
 Document(metadata={'source': '2008_Mumbai_attacks.pdf', 'page': 2}, page_content='Bullet holes on the wall at the CSMT\nA group of men said to number between 24 and 26[52] received training in marine warfare at a remote\ncamp in mountainous Muzaffarabad in Pakistan. Part of the training was said to have taken place on the\nMangla Da

In [16]:
from operator import itemgetter

chain = (
  {
    "context" : itemgetter("question") | retriever,
    "question" : itemgetter("question")
  }
  | prompt
  | model
  | parser
)

# for response in chain.stream({ "question": "what are the series of events in the attack" }):
#   print(response)
print(chain.invoke({ "question": "tell me about the cafe that was attacked" }))

The Leopold Cafe, a popular restaurant and bar on Colaba Causeway in South Mumbai, was one of the first sites to be attacked. Two attackers opened fire on the cafe on the evening of 26 November, killing 10 people (including some international visitors) and injuring many more.  

